In [1]:
import numpy as np
from glob import glob
from src.model import *
from src.utils import *
import tensorflow as tf
from src.cityscapes import *
from datetime import datetime
from matplotlib import pyplot as plt
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler

In [2]:
batch_size = 12

train_dataset = get_dataset(
    glob('./data/train_imgs/*'),
    glob('./data/train_labels/*')
)

val_dataset = get_dataset(
    glob('./data/val_imgs/*'),
    glob('./data/val_labels/*')
)

Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.


In [3]:
print(train_dataset)
print(val_dataset)

<PrefetchDataset shapes: ((12, 2048, 1024, 3), (12, 2048, 1024, 1)), types: (tf.float32, tf.uint8)>
<PrefetchDataset shapes: ((12, 2048, 1024, 3), (12, 2048, 1024, 1)), types: (tf.float32, tf.uint8)>


In [4]:
loss = SparseCategoricalCrossentropy(from_logits=True)
strategy = tf.distribute.MirroredStrategy()

In [5]:
with strategy.scope():
    model = FastSCNN(n_classes=19)
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.SeparableConv2D):
            layer.kernel_regularizer = tf.keras.regularizers.l2(4e-5)
    lr_scheduler = PolynomialDecay(0.045, 1000, 1e-4, 0.9)
    optimizer = tf.keras.optimizers.SGD(momentum=0.9, learning_rate=lr_scheduler)
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [6]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = TensorBoard(log_dir=logdir, write_graph=True, update_freq='batch')
model_checkpoint = ModelCheckpoint(
    mode='min', filepath='fast_scnn_weights.h5',
    monitor='val_loss', save_best_only='True',
    save_weights_only='True', verbose=1
)

In [7]:
model.fit(
    train_dataset, validation_data=val_dataset, epochs=300,
    steps_per_epoch=len(glob('./data/train_imgs/*')) // batch_size,
    validation_steps=len(glob('./data/val_imgs/*')) // batch_size,
    callbacks=[tensorboard, model_checkpoint]
)

Train for 247 steps, validate for 41 steps
Epoch 1/300
  1/247 [..............................] - ETA: 2:00:28WARNING:tensorflow:Can save best model only with val_loss available, skipping.


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[12,2048,1024,19] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node Fast-SCNN/softmax/Softmax (defined at /local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[metrics/accuracy/div_no_nan/ReadVariableOp/_2]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[12,2048,1024,19] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node Fast-SCNN/softmax/Softmax (defined at /local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1751